In [143]:
import numpy as np
import pandas as pd


In [144]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [145]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [186]:
movies = movies.merge(credits, on='title')

In [187]:
#dorkar ase amar poject a
#genres
#id 
#keyword
#title
#overview
#cast
#crew
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [153]:
movies.duplicated().sum()

np.int64(0)

In [188]:
movies.dropna(inplace=True)

In [155]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [156]:
movies['genres'] = movies['genres'].apply(convert)

In [157]:
movies['keywords'] = movies['keywords'].apply(convert)

In [158]:
import ast

def convert3(obj):
    L = []
    cnt = 0
    for i in ast.literal_eval(obj):
        if cnt != 3:
            L.append(i['name'])
            cnt += 1
        else:
            break
    return L


In [159]:
movies['cast']=movies['cast'].apply(convert3)

In [161]:
import ast

def find_director(obj):
    L = []
    cnt = 0
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L


In [162]:
movies['crew'] = movies['crew'].apply(find_director)

In [163]:
movies['overview'] = movies['overview'].apply(lambda x : x.split())

In [164]:
movies['crew'] = movies['crew'].apply(lambda x : [i.replace(" " ,"") for i in x])
movies['genres'] = movies['genres'].apply(lambda x : [i.replace(" " ,"") for i in x])
movies['cast'] = movies['cast'].apply(lambda x : [i.replace(" " ,"") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x : [i.replace(" " ,"") for i in x])

In [165]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4806 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4806 non-null   int64 
 1   title     4806 non-null   object
 2   overview  4806 non-null   object
 3   genres    4806 non-null   object
 4   keywords  4806 non-null   object
 5   cast      4806 non-null   object
 6   crew      4806 non-null   object
dtypes: int64(1), object(6)
memory usage: 300.4+ KB


In [166]:
movies['tags'] = movies['crew'] + movies['genres'] + movies['cast'] + movies['keywords'] + movies['overview']

In [168]:
df = movies[['movie_id', 'title','tags']]

In [170]:
df['tags'] = df['tags'].apply(lambda x : " ".join(x) )

C:\Users\HP\AppData\Local\Temp\ipykernel_17292\55382636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x : " ".join(x) )


In [171]:
df['tags'] = df['tags'].apply(lambda x : x.lower()) 

C:\Users\HP\AppData\Local\Temp\ipykernel_17292\2309949301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x : x.lower())


In [172]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')


In [173]:
vectors = cv.fit_transform(df['tags']).toarray()

In [174]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [175]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [176]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [177]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [178]:
df['tags'] = df['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_17292\866399325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [179]:
df

,movie_id,title,tags
0,19995,Avatar,jamescameron action adventur fantasi sciencefi...
1,285,Pirates of the Caribbean: At World's End,goreverbinski adventur fantasi action johnnyde...
2,206647,Spectre,sammend action adventur crime danielcraig chri...
3,49026,The Dark Knight Rises,christophernolan action crime drama thriller c...
4,49529,John Carter,andrewstanton action adventur sciencefict tayl...
...,...,...,...
4804,9367,El Mariachi,robertrodriguez action crime thriller carlosga...
4805,72766,Newlyweds,edwardburn comedi romanc edwardburn kerrybishé...
4806,231617,"Signed, Sealed, Delivered",scottsmith comedi drama romanc tvmovi ericmabi...
4807,126186,Shanghai Calling,danielhsia danielhenney elizacoup billpaxton w...


In [180]:
from sklearn.metrics.pairwise import cosine_similarity

In [181]:
similarity = cosine_similarity(vectors)

In [182]:
df[df['title'] == 'Batman Begins'].index

Index([119], dtype='int64')

In [183]:
def recommend(movie):
    movie_index = df[df['title'] == movie].index[0]
    distance = similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)) , reverse = True , key = lambda x : x[1])[1:6]
    for i in movies_list:
        print(df.iloc[i[0]].title)
    return

In [184]:
recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman
Batman & Robin
